In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col 

In [0]:
spark = SparkSession.builder.appName("this is my first program in spark").getOrCreate()

In [0]:
data = spark.read.csv("dbfs:/FileStore/phone_usage_india.csv",header=True,inferSchema=True)

In [0]:
data.select('Location').distinct().count()

Out[4]: 10

In [0]:
data.select('OS').distinct().count()

Out[5]: 2

In [0]:
data.write.partitionBy('Location').parquet("dbfs:/FileStore/locationbydata")

In [0]:
data.write.partitionBy('OS').parquet("dbfs:/FileStore/OS_based_data")

In [0]:
data = spark.read.parquet('dbfs:/FileStore/locationbydata')

In [0]:
data.filter(data['Location'] == "Mumbai").show()

+-------+---+------+------------+-------+---------------------+---------------------+-------------------------+------------------------+---------------------------+----------------------------+------------------------+---------------------+---------------------------+-------------+--------+
|User ID|Age|Gender| Phone Brand|     OS|Screen Time (hrs/day)|Data Usage (GB/month)|Calls Duration (mins/day)|Number of Apps Installed|Social Media Time (hrs/day)|E-commerce Spend (INR/month)|Streaming Time (hrs/day)|Gaming Time (hrs/day)|Monthly Recharge Cost (INR)|  Primary Use|Location|
+-------+---+------+------------+-------+---------------------+---------------------+-------------------------+------------------------+---------------------------+----------------------------+------------------------+---------------------+---------------------------+-------------+--------+
| U00001| 53|  Male|        Vivo|Android|                  3.7|                 23.9|                     37.9|             

In [0]:
data.filter(data['Location'] == "Mumbai").groupBy('Location').agg({"Screen Time (hrs/day)":"avg"}).show()

+--------+--------------------------+
|Location|avg(Screen Time (hrs/day))|
+--------+--------------------------+
|  Mumbai|        6.4210220673635305|
+--------+--------------------------+



In [0]:
from pyspark.sql.functions import broadcast

In [0]:
data = spark.read.csv("dbfs:/FileStore/phone_usage_india.csv",header=True,inferSchema=True)

In [0]:
data1 = spark.read.csv("dbfs:/FileStore/phone_usage_india.csv",header=True,inferSchema=True)
data1 = data1.withColumnRenamed("Screen Time (hrs/day)",'Screen Time')
data1 = data1.withColumnRenamed("Location",'location1')

In [0]:
data.join(broadcast(data1),data['Phone Brand'] == data1['Phone Brand'],"inner").show()

+-------+---+------+--------+-----------+-------+---------------------+---------------------+-------------------------+------------------------+---------------------------+----------------------------+------------------------+---------------------+---------------------------+-----------+-------+---+------+---------+-----------+-------+-----------+---------------------+-------------------------+------------------------+---------------------------+----------------------------+------------------------+---------------------+---------------------------+-------------+
|User ID|Age|Gender|Location|Phone Brand|     OS|Screen Time (hrs/day)|Data Usage (GB/month)|Calls Duration (mins/day)|Number of Apps Installed|Social Media Time (hrs/day)|E-commerce Spend (INR/month)|Streaming Time (hrs/day)|Gaming Time (hrs/day)|Monthly Recharge Cost (INR)|Primary Use|User ID|Age|Gender|location1|Phone Brand|     OS|Screen Time|Data Usage (GB/month)|Calls Duration (mins/day)|Number of Apps Installed|Social Me

In [0]:
data_joinied = data.join(broadcast(data1),data['Phone Brand'] == data1['Phone Brand'],"inner")

In [0]:
data_joinied.groupBy('Location').agg({"Screen Time (hrs/day)":"avg"}).show()

+---------+--------------------------+
| Location|avg(Screen Time (hrs/day))|
+---------+--------------------------+
|Bangalore|         6.575263746695015|
|  Lucknow|          6.57994686386275|
|  Chennai|         6.529346644202747|
|   Mumbai|         6.421639501550293|
|Ahmedabad|         6.563978713538523|
|  Kolkata|         6.412582819510596|
|     Pune|        6.6679595900518045|
|    Delhi|         6.469169119614471|
|Hyderabad|         6.593114127383831|
|   Jaipur|         6.654786788313208|
+---------+--------------------------+



In [0]:
data_joinied.columns

Out[42]: ['User ID',
 'Age',
 'Gender',
 'Location',
 'Phone Brand',
 'OS',
 'Screen Time (hrs/day)',
 'Data Usage (GB/month)',
 'Calls Duration (mins/day)',
 'Number of Apps Installed',
 'Social Media Time (hrs/day)',
 'E-commerce Spend (INR/month)',
 'Streaming Time (hrs/day)',
 'Gaming Time (hrs/day)',
 'Monthly Recharge Cost (INR)',
 'Primary Use',
 'User ID',
 'Age',
 'Gender',
 'location1',
 'Phone Brand',
 'OS',
 'Screen Time',
 'Data Usage (GB/month)',
 'Calls Duration (mins/day)',
 'Number of Apps Installed',
 'Social Media Time (hrs/day)',
 'E-commerce Spend (INR/month)',
 'Streaming Time (hrs/day)',
 'Gaming Time (hrs/day)',
 'Monthly Recharge Cost (INR)',
 'Primary Use']

In [0]:
data.show()

+-------+---+------+---------+------------+-------+---------------------+---------------------+-------------------------+------------------------+---------------------------+----------------------------+------------------------+---------------------+---------------------------+-------------+
|User ID|Age|Gender| Location| Phone Brand|     OS|Screen Time (hrs/day)|Data Usage (GB/month)|Calls Duration (mins/day)|Number of Apps Installed|Social Media Time (hrs/day)|E-commerce Spend (INR/month)|Streaming Time (hrs/day)|Gaming Time (hrs/day)|Monthly Recharge Cost (INR)|  Primary Use|
+-------+---+------+---------+------------+-------+---------------------+---------------------+-------------------------+------------------------+---------------------------+----------------------------+------------------------+---------------------+---------------------------+-------------+
| U00001| 53|  Male|   Mumbai|        Vivo|Android|                  3.7|                 23.9|                     37.9|

In [0]:
data = spark.read.csv("dbfs:/FileStore/phone_usage_india.csv",header=True,inferSchema=True).select("Gender","Screen Time (hrs/day)","Social Media Time (hrs/day)")

In [0]:
data.show()

+------+---------------------+---------------------------+
|Gender|Screen Time (hrs/day)|Social Media Time (hrs/day)|
+------+---------------------+---------------------------+
|  Male|                  3.7|                        3.9|
| Other|                  9.2|                        2.8|
|Female|                  4.5|                        3.0|
|  Male|                 11.0|                        5.2|
|  Male|                  2.2|                        5.5|
|  Male|                  5.4|                        4.2|
|Female|                  6.0|                        0.8|
| Other|                  3.1|                        2.3|
|Female|                  5.3|                        3.7|
| Other|                  9.9|                        4.8|
| Other|                  1.6|                        3.1|
|Female|                  7.5|                        0.9|
|  Male|                 10.5|                        4.5|
|  Male|                 10.3|                        5.

In [0]:
data.explain(True)

== Parsed Logical Plan ==
'Project ['Gender, 'Screen Time (hrs/day), 'Social Media Time (hrs/day)]
+- Relation [User ID#3033,Age#3034,Gender#3035,Location#3036,Phone Brand#3037,OS#3038,Screen Time (hrs/day)#3039,Data Usage (GB/month)#3040,Calls Duration (mins/day)#3041,Number of Apps Installed#3042,Social Media Time (hrs/day)#3043,E-commerce Spend (INR/month)#3044,Streaming Time (hrs/day)#3045,Gaming Time (hrs/day)#3046,Monthly Recharge Cost (INR)#3047,Primary Use#3048] csv

== Analyzed Logical Plan ==
Gender: string, Screen Time (hrs/day): double, Social Media Time (hrs/day): double
Project [Gender#3035, Screen Time (hrs/day)#3039, Social Media Time (hrs/day)#3043]
+- Relation [User ID#3033,Age#3034,Gender#3035,Location#3036,Phone Brand#3037,OS#3038,Screen Time (hrs/day)#3039,Data Usage (GB/month)#3040,Calls Duration (mins/day)#3041,Number of Apps Installed#3042,Social Media Time (hrs/day)#3043,E-commerce Spend (INR/month)#3044,Streaming Time (hrs/day)#3045,Gaming Time (hrs/day)#3046,